In [ ]:
!rm -r Deepfashion2_Training/

In [ ]:
!git clone https://github.com/ketnas/Deepfashion2_Training.git

Cloning into 'Deepfashion2_Training'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 110 (delta 6), reused 7 (delta 3), pack-reused 98
Receiving objects: 100% (110/110), 74.80 MiB | 31.89 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 1.x selected.
1.15.2
Found GPU at: /device:GPU:0


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!curl https://sdk.cloud.google.com | bash

In [ ]:
!gcloud init

In [ ]:
!gsutil ls

gs://fashion_test/


In [ ]:
!mkdir Dataset

In [ ]:
!gsutil cp gs://fashion_test/train.zip ./Dataset
!gsutil cp gs://fashion_test/validation.zip ./Dataset

Copying gs://fashion_test/train.zip...
\ [1 files][  9.9 GiB/  9.9 GiB]   39.3 MiB/s                                   
Operation completed over 1 objects/9.9 GiB.                                      
Copying gs://fashion_test/validation.zip...
- [1 files][  1.7 GiB/  1.7 GiB]   11.0 MiB/s                                   
Operation completed over 1 objects/1.7 GiB.                                      


In [ ]:
!unzip -P 2019Deepfashion2** ./Dataset/train.zip
!unzip -P 2019Deepfashion2** ./Dataset/validation.zip

In [ ]:
!mv ./train ./Dataset

In [ ]:
!mv ./validation ./Dataset

In [ ]:
!gsutil cp gs://fashion_test/train.json ./Dataset/train
!gsutil cp gs://fashion_test/valid.json ./Dataset/validation

Copying gs://fashion_test/train.json...
\ [1 files][  1.5 GiB/  1.5 GiB]   44.0 MiB/s                                   
Operation completed over 1 objects/1.5 GiB.                                      
Copying gs://fashion_test/valid.json...
/ [1 files][386.7 MiB/386.7 MiB]                                                
Operation completed over 1 objects/386.7 MiB.                                    


**Start from training from here**

In [ ]:
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

import sys
sys.dont_write_bytecode = True

import os
import numpy as np

from pycocotools.coco import COCO
from pycocotools import mask as maskUtils
from Deepfashion2_Training.lib.config import Config
from Deepfashion2_Training.lib.model import MaskRCNN
from Deepfashion2_Training.lib import utils

%matplotlib inline

ROOT_DIR = os.path.abspath("./")

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

Using TensorFlow backend.


In [ ]:
class DeepFashion2Config(Config):
    """Configuration for training on DeepFashion2.
    Derives from the base Config class and overrides values specific
    to the DeepFashion2 dataset.
    """
    # Give the configuration a recognizable name
    NAME = "deepfashion2"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Uncomment to train on 8 GPUs (default is 1)
    #GPU_COUNT = 4
    GPU_COUNT = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 13  # COCO has 80 classes
    
    USE_MINI_MASK = True

    train_img_dir = "./Dataset/train/image"
    train_json_path = "./Dataset/train/train.json"
    valid_img_dir = "./Dataset/validation/image"
    valid_json_path = "./Dataset/validation/valid.json"


############################################################
#  Dataset
############################################################

class DeepFashion2Dataset(utils.Dataset):
    def load_coco(self, image_dir, json_path, class_ids=None,
                  class_map=None, return_coco=False):
        """Load the DeepFashion2 dataset.
        """

        coco = COCO(json_path)

        # Load all classes or a subset?
        if not class_ids:
            # All classes
            class_ids = sorted(coco.getCatIds())

        # All images or a subset?
        if class_ids:
            image_ids = []
            for id in class_ids:
                image_ids.extend(list(coco.getImgIds(catIds=[id])))
            # Remove duplicates
            image_ids = list(set(image_ids))
        else:
            # All images
            image_ids = list(coco.imgs.keys())

        # Add classes
        for i in class_ids:
            self.add_class("deepfashion2", i, coco.loadCats(i)[0]["name"])

        # Add images
        for i in image_ids:
            self.add_image(
                "deepfashion2", image_id=i,
                path=os.path.join(image_dir, coco.imgs[i]['file_name']),
                width=coco.imgs[i]["width"],
                height=coco.imgs[i]["height"],
                annotations=coco.loadAnns(coco.getAnnIds(
                    imgIds=[i], catIds=class_ids, iscrowd=None)))
        if return_coco:
            return coco

    def load_keypoint(self, image_id):
        """
        """
        image_info = self.image_info[image_id]
        if image_info["source"] != "deepfashion2":
            return super(DeepFashion2Dataset, self).load_mask(image_id)

        instance_keypoints = []
        class_ids = []
        annotations = self.image_info[image_id]["annotations"]

        for annotation in annotations:
            class_id = self.map_source_class_id(
                "deepfashion2.{}".format(annotation['category_id']))
            if class_id:
                keypoint = annotation['keypoints']

                instance_keypoints.append(keypoint)
                class_ids.append(class_id)

        keypoints = np.stack(instance_keypoints, axis=1)
        class_ids = np.array(class_ids, dtype=np.int32)
        return keypoints, class_ids
            
    def load_mask(self, image_id):
        """Load instance masks for the given image.
        Different datasets use different ways to store masks. This
        function converts the different mask format to one format
        in the form of a bitmap [height, width, instances].
        Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a COCO image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "deepfashion2":
            return super(DeepFashion2Dataset, self).load_mask(image_id)

        instance_masks = []
        class_ids = []
        annotations = self.image_info[image_id]["annotations"]
        # Build mask of shape [height, width, instance_count] and list
        # of class IDs that correspond to each channel of the mask.
        for annotation in annotations:
            class_id = self.map_source_class_id(
                "deepfashion2.{}".format(annotation['category_id']))
            if class_id:
                m = self.annToMask(annotation, image_info["height"],
                                   image_info["width"])
                # Some objects are so small that they're less than 1 pixel area
                # and end up rounded out. Skip those objects.
                if m.max() < 1:
                    continue
                # Is it a crowd? If so, use a negative class ID.
                if annotation['iscrowd']:
                    # Use negative class ID for crowds
                    class_id *= -1
                    # For crowd masks, annToMask() sometimes returns a mask
                    # smaller than the given dimensions. If so, resize it.
                    if m.shape[0] != image_info["height"] or m.shape[1] != image_info["width"]:
                        m = np.ones([image_info["height"], image_info["width"]], dtype=bool)
                instance_masks.append(m)
                class_ids.append(class_id)

        # Pack instance masks into an array
        if class_ids:
            mask = np.stack(instance_masks, axis=2).astype(np.bool)
            class_ids = np.array(class_ids, dtype=np.int32)
            return mask, class_ids
        else:
            # Call super class to return an empty mask
            return super(DeepFashion2Dataset, self).load_mask(image_id)

    def image_reference(self, image_id):
        """Return a link to the image in the COCO Website."""
        super(DeepFashion2Dataset, self).image_reference(image_id)

    # The following two functions are from pycocotools with a few changes.

    def annToRLE(self, ann, height, width):
        """
        Convert annotation which can be polygons, uncompressed RLE to RLE.
        :return: binary mask (numpy 2D array)
        """
        segm = ann['segmentation']
        if isinstance(segm, list):
            # polygon -- a single object might consist of multiple parts
            # we merge all parts into one mask rle code
            rles = maskUtils.frPyObjects(segm, height, width)
            rle = maskUtils.merge(rles)
        elif isinstance(segm['counts'], list):
            # uncompressed RLE
            rle = maskUtils.frPyObjects(segm, height, width)
        else:
            # rle
            rle = ann['segmentation']
        return rle

    def annToMask(self, ann, height, width):
        """
        Convert annotation which can be polygons, uncompressed RLE, or RLE to binary mask.
        :return: binary mask (numpy 2D array)
        """
        rle = self.annToRLE(ann, height, width)
        m = maskUtils.decode(rle)
        return m


def train(model, config):
    """
    """
    dataset_train = DeepFashion2Dataset()
    dataset_train.load_coco(config.train_img_dir, config.train_json_path)
    dataset_train.prepare()

    dataset_valid = DeepFashion2Dataset()
    dataset_valid.load_coco(config.valid_img_dir, config.valid_json_path)
    dataset_valid.prepare()

    model.train(dataset_train, dataset_valid,
                learning_rate=config.LEARNING_RATE,
                epochs=1,
                layers='heads')


In [ ]:
config = DeepFashion2Config()
model = MaskRCNN(mode="training", config=config,
                                  model_dir=MODEL_DIR)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


In [ ]:
model.load_weights(COCO_MODEL_PATH, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                                        "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
train(model, config)

loading annotations into memory...


In [ ]:
model_path = os.path.join(MODEL_DIR, "mask_rcnn_deepfashion1.h5")
model.keras_model.save_weights(model_path)

NameError: ignored

In [ ]:
!wc -l ./Dataset/train/train.json

0 ./Dataset/train/train.json


In [ ]:
!head -1 ./Dataset/train/train.json